In [1]:
!git clone https://github.com/FedorZaitsev/VKR25
%cd VKR25

Cloning into 'VKR25'...
remote: Enumerating objects: 158, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 158 (delta 61), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (158/158), 1.38 MiB | 4.47 MiB/s, done.
Resolving deltas: 100% (61/61), done.
/kaggle/working/VKR25


In [2]:
import os

config = {
    'SEED' : 228,
    
    'BOS_TOKEN' : 4096,
    'EOS_TOKEN' : 4097,
    'INP_PAD_TOKEN' : 4098,
    'TAR_PAD_TOKEN' : -100,
    'VOCAB_SIZE' : 4099,
    'MAX_LENGTH' : 256,
    'OVERLAP' : 64,
    
    'NUM_WORKERS' : 4,
    'BATCH_SIZE' : 16,

    'ACCUM_STEPS' : 1,
}

for key, value in config.items():
    os.environ[key] = str(value)

In [3]:
import torch
import random
import numpy as np

device=torch.device('cuda')
root_dir = '/kaggle/input/groove-tokens'

torch.manual_seed(config['SEED'])
random.seed(config['SEED'])
np.random.seed(config['SEED'])

In [4]:
from models.transformer_model import TransformerModel

In [5]:
model = torch.load('/kaggle/input/huge_transf/pytorch/default/1/checkpoint_400.pt', map_location=device)

/tmp/ipykernel_19/3560529179.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/kaggle/input/huge_transf/pytorch/default/1/checkpoint_400.pt', map_loca

In [6]:
from tqdm import tqdm

BOS_TOKEN = 4096
EOS_TOKEN = 4097
INP_PAD_TOKEN = 4098

sequence = [BOS_TOKEN]

def generate(model, seq=sequence, max_len=750, tmp=1.0, force=False, watch_tail=256):

    if watch_tail is None:
        watch_tail = 256
    watch_tail = min(256, watch_tail)
    
    device = next(model.parameters()).device
    model.eval()
    generated = seq.copy()

    with torch.no_grad():
        for _ in tqdm(range(max_len - len(seq))):

            src = torch.tensor(generated, dtype=torch.long).unsqueeze(0).to(device)
            if watch_tail is not None:
                src = src[:, -watch_tail:]

            mask = model.generate_square_subsequent_mask(src.shape[1]).to(device)
            output = model(src, mask)
            logits = output[0, -1, :] / tmp
            token = torch.distributions.categorical.Categorical(logits=logits).sample()

            if token.item() > 4095:
                if not force:
                    break
                else:
                    token = torch.distributions.categorical.Categorical(logits=logits[:-3]).sample()
            generated.append(token.item())


    return generated[1:]

In [7]:
!git clone https://github.com/jishengpeng/WavTokenizer
%cd WavTokenizer

Cloning into 'WavTokenizer'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 200 (delta 65), reused 37 (delta 35), pack-reused 84 (from 1)
Receiving objects: 100% (200/200), 469.31 KiB | 11.73 MiB/s, done.
Resolving deltas: 100% (83/83), done.
/kaggle/working/VKR25/WavTokenizer


In [8]:
!wget https://huggingface.co/novateur/WavTokenizer-medium-music-audio-75token/resolve/main/wavtokenizer_medium_music_audio_320_24k_v2.ckpt
!wget https://huggingface.co/novateur/WavTokenizer-medium-music-audio-75token/resolve/main/wavtokenizer_mediumdata_music_audio_frame75_3s_nq1_code4096_dim512_kmeans200_attn.yaml

--2025-05-13 10:19:05--  https://huggingface.co/novateur/WavTokenizer-medium-music-audio-75token/resolve/main/wavtokenizer_medium_music_audio_320_24k_v2.ckpt
Resolving huggingface.co (huggingface.co)... 18.239.50.49, 18.239.50.16, 18.239.50.80, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.49|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/f8/d8/f8d8b97f33126a1e3a4c3ffe2e6af86c26776cfa33aee1294672329b62115562/078d11581aa10cc91572bfcff7ef00b71d8e24c4c359e98f9194a1a9d21ae8a8?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27wavtokenizer_medium_music_audio_320_24k_v2.ckpt%3B+filename%3D%22wavtokenizer_medium_music_audio_320_24k_v2.ckpt%22%3B&Expires=1747135145&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzEzNTE0NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2Y4L2Q4L2Y4ZDhiOTdmMzMxMjZhMWUzYTRjM2ZmZTJlNmFmODZjMjY3NzZjZmEzM2FlZTEyOTQ2NzIzM

In [9]:
from encoder.utils import convert_audio
import torchaudio
import torch
from decoder.pretrained import WavTokenizer

config_path = "/kaggle/working/VKR25/WavTokenizer/wavtokenizer_mediumdata_music_audio_frame75_3s_nq1_code4096_dim512_kmeans200_attn.yaml"
model_path = "/kaggle/working/VKR25/WavTokenizer/wavtokenizer_medium_music_audio_320_24k_v2.ckpt"

wavtokenizer = WavTokenizer.from_pretrained0802(config_path, model_path)
wavtokenizer = wavtokenizer.to(device)

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/kaggle/working/VKR25/WavTokenizer/decoder/pretrained.py:101: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_onl

In [10]:
def tokens_to_wav(token_seq, name='example.wav'):
    audio_tokens = torch.tensor([token_seq], device=device)
    features = wavtokenizer.codes_to_features(audio_tokens)
    bandwidth_id = torch.tensor([0], device=device)
    audio_out = wavtokenizer.decode(features, bandwidth_id=bandwidth_id)
    torchaudio.save(name, audio_out.cpu(), sample_rate=24000, encoding='PCM_S', bits_per_sample=16)

In [11]:
!mkdir /kaggle/working/VKR25/WavTokenizer/wavs

In [12]:
for i in range(1000):
    ans1 = generate(model, tmp=1, force=True, watch_tail=240)
    tokens_to_wav(ans1, f'wavs/sample{i}.wav')

100%|██████████| 749/749 [00:03<00:00, 199.06it/s]


In [13]:
import shutil
shutil.make_archive('wavs', 'zip', '/kaggle/working/VKR25/WavTokenizer/wavs')

'/kaggle/working/VKR25/WavTokenizer/wavs.zip'